In [2]:
%reload_ext autoreload
%autoreload 2

In [133]:
from padding_oracle import CBCPaddingOracle
import requests

def oracle(iv: bytes, block: bytes):
    # Create token with the iv and block
    token = iv + block

    # Create request and attach cookie with token
    cookie = {"authtoken": token.hex()}
    r = requests.get("http://localhost:5000/quote", cookies=cookie)
    # 
    # Check response
    if (r.content == b'Padding is incorrect.' or r.content == b'PKCS#7 padding is incorrect.' or  r.content == b'Zero-length input cannot be unpadded'): #r.content == b'No quote for you!' or
        return False
    else:
        return True


# Establish baseline
r = requests.get("http://localhost:5000")
ciphertext_str = r.cookies["authtoken"]
ciphertext_bytes = bytearray.fromhex(ciphertext_str)

breaker = CBCPaddingOracle(ciphertext_bytes, 16, oracle)
print(breaker.attack())

Block 1:           a7ad7ffefef6177545290cd3a7e75eb3                                                                                                                                                                         
DEC-Block 1:       b7bd6feeeee6076555391cc3b7f74ea3
Plaintext-Block 1: 596f75206e6576657220666967757265
------------------------------------------------------------
Block 2:           87a51c815a2ab18c797b610d92d5975b                                                                                                                                                                         
DEC-Block 2:       97b50c914a3aa19c696b711d82c5874b
Plaintext-Block 2: 206f757420746861742022492073686f
------------------------------------------------------------
Block 3:           b5a6eed1bcdec12fc29bf145b74d6e5b                                                                                                                                                                         
DEC-Blo

b'You never figure out that "I should have used authenticated encryption because ...". :)\t\t\t\t\t\t\t\t\t'